# Objective

This tutorial session is devoted to learn how to simulate sensors within Gazebo_ros.

## Provided packages

```
git clone https://github.com/ggory15/ros_tutorials
```

or

```
git pull origin master
```

### Add Gazebo model
```
gedit ~/.bashrc
```

Then, 
```
export GAZEBO_MODEL_PATH={Your Workspace}/1027_tutorial/gazebo_sensor_tutorial/models/:${GAZEBO_MODEL_PATh}
```


## Sensors in Gazebo and ROS

Related to sensors, Gazebo provides:

1. The [gazebo_models repository](https://github.com/osrf/gazebo_models) with some SDF models of sensors that include geometric descriptions and sensor features, like the camera.
2. A set of [sensor classes](http://osrf-distributions.s3.amazonaws.com/gazebo/api/dev/group__gazebo__sensors.html), like [CameraSensor](http://osrf-distributions.s3.amazonaws.com/gazebo/api/dev/classgazebo_1_1sensors_1_1CameraSensor.html), that define the behavior of the sensors. All of them inherit from class Sensor.
3. A set of plugins, like the [camera plugin](http://osrf-distributions.s3.amazonaws.com/gazebo/api/dev/classgazebo_1_1CameraPlugin.html), that interface the corresponding sensor classes. They inherit from a general sensor plugin class that has access to the Sensor class.

Because Gazebo and ROS are separate projects that do not depend on each other, ROS sensor plugins have been implemented to wrap the Gazebo sensor plugins. The available ROS sensor plugins are available in the gazebo_plugins of gazebo_ros_pkgs, like those related to cameras and depth cameras.

Plugins can be added to SDF sensor models or to sensor models defined using URDF.

The cameras and depth camera sensors will be described next, their ROS plugins and their modeling using both SDF and URDF. Plugins for other sensors will be just listed in section ROS plugins for other sensors.

## The Camera Sensor
### The Camera ROS plugin

The camera ROS plugin provides the ROS interface for simulating cameras by publishing the sensor_msgs::CameraInfo and sensor_msgs::Image ROS messages.
The ROS camera plugin is libgazebo_ros_camera.so. Some implementation details are included in Section Plugin implementation.
To associate the plugin to a given camera model, the following XML code has to be included in the SDF or URDF model description:

```
<plugin name="camera_controller" filename="libgazebo_ros_camera.so">
  <alwaysOn>true</alwaysOn>
  <updateRate>0.0</updateRate>
  <cameraName>camera</cameraName>
  <imageTopicName>image_raw</imageTopicName>
  <cameraInfoTopicName>camera_info</cameraInfoTopicName>
  <frameName>camera_link_optical</frameName>
  <!-- setting hackBaseline to anything but 0.0 will cause a misalignment
           between the gazebo sensor image and the frame it is supposed to
           be attached to -->
  <hackBaseline>0.0</hackBaseline>
  <distortionK1>0.0</distortionK1>
  <distortionK2>0.0</distortionK2>
  <distortionK3>0.0</distortionK3>
  <distortionT1>0.0</distortionT1>
  <distortionT2>0.0</distortionT2>
  <CxPrime>0</CxPrime>
  <Cx>0.0</Cx>
  <Cy>0.0</Cy>
  <focalLength>0.0</focalLength>
</plugin>
```

The main arguments to be particularized are:

        <cameraName>: the camera name (camera)

        <imageTopicName>: the topic name of the advertised sensor_msgs::Image message (image_raw)

        <cameraInfoTopicName>: the topic name of the advertised sensor_msgs::CameraInfo message (camera_info)

        <frameName>: the coordinate frame the image is published under in the tf tree (camera_link_optical)

The <updateRate> is usually set to zero, which causes the plugin to publish information at the same rate as the parent SDF sensor (which is specified in the field <update_rate>, see below).


### Exersice 1

Take a look at the file model.sdf in the folder models/camera-plugin of the gazebo_sensors_tutorial package.

A launch file is provided to test this sensor model; it loads this camera and a coke can. The image message advertised can be visualized in rviz (that uses the camera_coke2.rviz configuration file where the Image Topic has been set to /camera/image_raw). The launch file to run rviz uses a static_transform_publisher to visualize the camera reference frame (called camera_link_optical):

```
roslaunch gazebo_sensors_tutorial camera_coke_gazebo2.launch
```
```
rostopic list
rostopic echo /camera/camera_info
roslaunch gazebo_sensors_tutorial camera_coke_rviz2.launch
```
![image1](./image/fig1a.png)
![image2](./image/fig1b.png)


### A URDF model

URDF robot models may contain description of sensors with the associated plugins. As an example, package rrbot_description includes in the rrbot.xacro file an end-effector link called camera_link associated to a camera. It contains all the visual and collision geometric descriptions (in this case a simple cube). Complementary, the rrbot.gazebo file contains an associated <gazebo> tag that includes the <sensor> element (that is directly passed to the <sdf> model) with the camera features and the plugin used with the following key information:

1. the camera name (rrbot/camera1)
2. the topic name of the advertised sensor_msgs::Image message (image_raw)
3. the topic name of the advertised sensor_msgs::CameraInfo message (camera_info)
4. the coordinate frame the image is published under in the tf tree (camera_link)

```
rence="camera_link">
   <sensor type="camera" name="camera1">
     <update_rate>30.0</update_rate>
     <camera name="head">
       <horizontal_fov>1.3962634</horizontal_fov>
       <image>
         <width>800</width>
         <height>800</height>
         <format>R8G8B8</format>
       </image>
       <clip>
         <near>0.02</near>
         <far>300</far>
       </clip>
       <noise>
         <type>gaussian</type>
         <!-- Noise is sampled independently per pixel on each frame.
              That pixel's noise value is added to each of its color
              channels, which at that point lie in the range [0,1]. -->
         <mean>0.0</mean>
         <stddev>0.007</stddev>
       </noise>
     </camera>
     <plugin name="camera_controller" filename="libgazebo_ros_camera.so">
       <alwaysOn>true</alwaysOn>
       <updateRate>0.0</updateRate>
       <cameraName>rrbot/camera1</cameraName>
       <imageTopicName>image_raw</imageTopicName>
       <cameraInfoTopicName>camera_info</cameraInfoTopicName>
       <frameName>camera_link_optical</frameName>
       <!-- setting hackBaseline to anything but 0.0 will cause a misalignment
           between the gazebo sensor image and the frame it is supposed to
           be attached to -->
       <hackBaseline>0.0</hackBaseline>
       <distortionK1>0.0</distortionK1>
       <distortionK2>0.0</distortionK2>
       <distortionK3>0.0</distortionK3>
       <distortionT1>0.0</distortionT1>
       <distortionT2>0.0</distortionT2>
       <CxPrime>0</CxPrime>
       <Cx>0.0</Cx>
       <Cy>0.0</Cy>
       <focalLength>0.0</focalLength>
     </plugin>
   </sensor>
 </gazebo>
```

### Exersice 2

The advertised images can be visualized using rviz. Follow the instruction below to load the rrbot and a coke can and to open rviz (in this case the Image Topic field is filled with the topic /rrbot/camera1/image_raw).


A launch file is provided to test this sensor model; it loads this camera and a coke can. The image message advertised can be visualized in rviz (that uses the camera_coke2.rviz configuration file where the Image Topic has been set to /camera/image_raw). The launch file to run rviz uses a static_transform_publisher to visualize the camera reference frame (called camera_link_optical):

```
roslaunch rrbot_gazebo rrbot_world_2.launch
rosrun gazebo_ros spawn_model -database coke_can -sdf -model coke_can -y 0.5 -x -1.2
roslaunch rrbot_description rrbot_rviz_2.launch
```
![image3](./image/fig2.png)


## The depth camera sensor
### The depth camera ROS plugin

The depth camera ROS plugin simulates a depth sensor like an Xbox-Kinect. The depth camera ROS plugin provides the ROS interface for simulating depth cameras by publishing the sensor_msgs::CameraInfo, sensor_msgs::Image and sensor_msgs::PointCloud2 ROS messages.

The ROS depth camera plugin is libgazebo_ros_openni_kinect.so. Some implementation details are included in Section Plugin implementation.

To associate the plugin to a given depth camera model, the following XML code has to be included in the SDF or URDF model description:

```
<plugin name="camera_plugin" filename="libgazebo_ros_openni_kinect.so">
  <baseline>0.2</baseline>
  <alwaysOn>true</alwaysOn>
  <!-- Keep this zero, update_rate in the parent <sensor> tag
       will control the frame rate. -->
  <updateRate>20.0</updateRate>
  <cameraName>camera_ir</cameraName>
  <imageTopicName>/camera/color/image_raw</imageTopicName>
  <cameraInfoTopicName>/camera/color/camera_info</cameraInfoTopicName>
  <depthImageTopicName>/camera/depth/image_raw</depthImageTopicName>
  <depthImageCameraInfoTopicName>/camera/depth/camera_info</depthImageCameraInfoTopicName>
  <pointCloudTopicName>/camera/depth/points</pointCloudTopicName>
  <frameName>depth_camera_link</frameName>
  <pointCloudCutoff>0.05</pointCloudCutoff>
  <distortionK1>0</distortionK1>
  <distortionK2>0</distortionK2>
  <distortionK3>0</distortionK3>
  <distortionT1>0</distortionT1>
  <distortionT2>0</distortionT2>
  <CxPrime>0</CxPrime>
  <Cx>0</Cx>
  <Cy>0</Cy>
  <focalLength>0</focalLength>
  <hackBaseline>0</hackBaseline>
</plugin>
```

The main arguments to be particularized are:

        the camera name (camera_ir)

        the name of the image topic (sensor_msgs::Image message /camera/color/image_raw)

        the name of the camera info topic (sensor_msgs::CameraInfo message /camera/color/camera_info)

        the name of the depth image topic (sensor_msgs::Image message /camera/depth/image_raw)

        the name of the depth camera info (sensor_msgs::CameraInfo message /camera/depth/camera_info) - this is the same as the camera info topic, but is only advertised when someone is subcribed to the other depth topics.

        the name of the point cloud topic (sensor_msgs::PointCloud2 message /camera/depth/points)

        the coordinate frame the data is published under in the tf tree (depth_camera_link)




### Exersice 3

Take a look at the file model.sdf in the folder models/kinect-plugin of the gazebo_sensors_tutorial package.

A launch file is provided to test this sensor model; it loads this kinect sensor and a coke can. The image message advertised can be visualized in rviz:

```
roslaunch gazebo_sensors_tutorial kinect_coke_gazebo2.launch
```
```
roslaunch gazebo_sensors_tutorial kinect_coke_rviz.launch
```
![image1](./image/fig3a.png)
![image2](./image/fig3b.png)


### Exersice 4

Take a look at the following files in the urdf/sensors folder of the mastering_ros_robot_description_pkg package:

    xtion_pro_live.gazebo.xacro, where a xacro:macro is used to define the depth camera sensor and its plugin,

    xtion_pro_live.urdf.xacro, where a xacro:macro is used to define the camera geometry and reference frames


```
roslaunch seven_dof_arm_gazebo seven_dof_arm_with_rgbd_world.launch
rosrun gazebo_ros spawn_model -database coke_can -sdf -model coke_can -x 0.4
roslaunch gazebo_sensors_tutorial kinect_arm_rviz.launch
```
![image1](./image/fig4.png)
![image2](./image/fig4b.png)


## Using Sensors

### ArUco library
As an application example, the estimation of the poses of objects will be tackled using the ArUco library, an open-source library for fast camera pose estimation using squared markers, and the aruco_ros package that wraps it. There are two alternatives nodes:

1. The aruco_ros nodes of type single look for a given marker ID and if found publishes several messages with its information (e.g. a geometry_msgs/PoseStamped with the pose and a visualization_msgs/Marker with the rviz marker), and a tf added to the tf_tree.
2. The aruco_ros nodes of type marker_publisher publishes a message of type aruco_msgs/MarkerArray with the information of the IDs and poses of all the markers found.

### Exercise 5
Move the pose of the cubes using the gazebo GUI and check how the new poses are tracked in rviz.

```
roslaunch gazebo_sensors_tutorial camera_gazebo2.launch
rosrun gazebo_ros spawn_model -database aruco_cube -sdf -model aruco_cube  -x 0.11 -z 0.05
rosrun gazebo_ros spawn_model -database pawnB1 -sdf -model pawnB1 -x 0.1 -y -0.1 -z 0.02
```

Then run the following launch file that:
    1. Starts two ArUco nodes, tuned to detect each of the two markers by specifying:
        
        the marker ID,
        
        the marker size,
        
        the marker frame
        
        the reference frame

    2. A node that subscribes to the published marker poses and broadcasts a transform located at the center of the objects.
    3. Starts rviz to visualize the detected markers and all the transforms.

```
roslaunch gazebo_sensors_tutorial camera_aruco_cube_rviz-2.launch

```

![image1](./image/fig5a.png)

If there are many markers to be detected all of the same size, a node of type marker_publisher should be run. The aruco_msgs/MarkerArray will contain the information of the detecte markers, but no tf are broadcasted. To do so, the package aruco_broadcater has a node of type aruco_broadcaster that broadcasts a tf of any marker detected by the marker_publisher node.

This is illustrated in the following example:
```
roslaunch gazebo_sensors_tutorial camera_gazebo2_cubes.launch
roslaunch gazebo_sensors_tutorial camera_aruco_cube_rviz-aruco_broadcaster.launch
```

The aruco_broadcaster node is configured with a yaml file with information regarding: a) which markers should be published; b) the frame w.r.t. the arucos are referred; c) the prefix of the tf that will be used, e.g. aruco_.

```
  markerList: []

  # the frame w.r.t. the arucos are referred
  camera_frame: camera_link_optical

  # the prefix of the tf that will be used
  aruco_frame: aruco
```

### Final Example (Chess demo)

An example has been prepared with two D415 cameras and a chessboard and two chess pieces labeled with ArUco makers. The world reference frame is located at the center of the chessboard.

```
roslaunch gazebo_sensors_tutorial cameras_chessboard.launch
```